geotiff_snow.py                                                     <br>
                                                                    <br>
 This code converts the SWE (Snow Water Equivalent) variable in     <br>
   the AMSR-2 NRT daily snow product in HDF-EOS5 format to GeoTiff, <br>
   with the use of GIBS color scheme for the variable.              <br>
                                                                    <br>
 Input: AMSR-2 NRT Daily Snow file in N/S hemisphere EASE-Grids     <br>
 Output: 2 GeoTIFF files of N. and S. hemis. SWE, respectively.     <br>
                                                                    <br>
 Required Python packages:                                          <br>
     numpy, pyproj, gdal and GHRC's GIBScolors_utils                <br>
                                                                    <br>
 Note:                                                              <br>
   User can set <useGIBScolors> to True/False to elect use of GIBS  <br>
   color scheme or not.                                             <br>
   In the case of useGIBScolors=True, <gibsMapDir> the location     <br>
   of GIBS color scheme files needs to be provided                  <br>
                                                                    <br>
 By Y. WU of AMSR team @GHRC, Sep 2021                              <br>

In [ ]:
import os,sys
import numpy as np
from osgeo import gdal, osr
from pyproj import Proj, transform
from GIBScolors_utils import *

epsg ='3408' & '3409' for EASE-N and EASE-S, respectively (Lambert Azimuthal Equal Area)

In [ ]:
EASEnPrj4='+proj=laea +lat_0=90 +lon_0=0 +x_0=0 +y_0=0 +a=6371228 +b=6371228 +units=m +no_defs'
EASEsPrj4='+proj=laea +lat_0=-90 +lon_0=0 +x_0=0 +y_0=0 +a=6371228 +b=6371228 +units=m +no_defs'

file name 

In [ ]:
h5file = 'C:/Users/lwang/Documents/AMSR/data/AMSR_U2_L3_DailySnow_R02_20210622.he5'
fileParts =h5file.split('_')
tstr = fileParts[-1].split('.')[0]

 Option to use GIBS color scheme

In [ ]:
useGIBScolors = True
gibsMapDir='C:/Users/lwang/Documents/AMSR/colormaps/'  #<-- ends with '/' for colormap files parsing

Open hdf5 with GDAL and get data attributes

In [ ]:
hdf_ds = gdal.Open(h5file, gdal.GA_ReadOnly)
dslist = hdf_ds.GetSubDatasets()
ds1 = [s[0].split('//HDFEOS/GRIDS/')[-1] for s in dslist] # include lat, lon

 Pick hemisphere/grid to process<br>
   NOTE: have to put N. and S. hemisphere variables in separate files<br>
         as they are in different geogrids

In [ ]:
hems=['N','S']
for hem in hems:
    if(hem == 'N'): 
        polarPrj4 = EASEnPrj4
        hemIDF  = '_Northern'
    if(hem == 'S'): 
        polarPrj4 = EASEsPrj4
        hemIDF  = '_Southern'
    SIPrj = osr.SpatialReference()
    SIPrj.ImportFromProj4(polarPrj4)
    Pgrid = hemIDF[1:]+'_Hemisphere'
    vname = 'SWE'+hemIDF+'Daily'

    #---get (lat, lon) for the grid and geotransform info
    for ii, a in enumerate(ds1):
        if(Pgrid+'/lat' in a): ilat = ii
        if(Pgrid+'/lon' in a): ilon = ii
    Lat = gdal.Open(hdf_ds.GetSubDatasets()[ilat][0], gdal.GA_ReadOnly)
    Lon = gdal.Open(hdf_ds.GetSubDatasets()[ilon][0], gdal.GA_ReadOnly)
    Lat = Lat.ReadAsArray()[1:-1,1:-1]
    Lon = Lon.ReadAsArray()[1:-1,1:-1]
    NY, NX = Lon.shape
    projectn = Proj(polarPrj4)
    X,Y = projectn(Lon,Lat,inverse=False)
    xmin, ymin, xmax, ymax = [X.min(), Y.min(), X.max(), Y.max()]
    xres = (xmax - xmin) / float(NX-1)
    yres = (ymax - ymin) / float(NY-1)
    geotransform = (xmin, xres, 0, ymax, 0, -yres)

    #---get data from the original hdf dataset
    for ii, a in enumerate(ds1):
        if(vname in a): isubset = ii
    varband = gdal.Open(hdf_ds.GetSubDatasets()[isubset][0], gdal.GA_ReadOnly)
    bandArr = varband.ReadAsArray()
    bandArr = bandArr[1:-1, 1:-1]  #remove outer layer cells

    #-----------------------------------------------
    #- set driver, projection, geotransformation   -
    #-  for output geotiff                         -
    #-----------------------------------------------
    #<--- output geotiff file name with dir path
    TIFFpath= 'C:/Users/lwang/Documents/AMSR/Data_Out/'+vname+'_'+tstr+'.tif'
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(TIFFpath,
                           NX, NY,
                           1,
                           gdal.GDT_Byte)
    out_ds.SetProjection(SIPrj.ExportToWkt())
    out_ds.SetGeoTransform(geotransform)

    #---activate output band
    outBand = out_ds.GetRasterBand(1)
    notVal = 255 # same as fill_value in original file

    #---If opted, use GIBS colors to set colortable
    scaleFC = {}
    if(useGIBScolors): 
        gibsColors = gibsScheme(gibsMapDir, 'DailySnow', vname, scaleFC)
        if gibsColors:
            colors = colorTable(gibsColors, 255, scaleFC, verb=False)
            outBand.SetColorTable(colors)
            outBand.SetColorInterpretation(gdal.GCI_PaletteIndex)

    #---set off-earth data to notValue, too
    bandArr[bandArr==248] = notVal

    #---wrting geotiff band
    outBand.WriteArray(bandArr)
    outBand.SetDescription(vname)
    outBand.SetNoDataValue(notVal)
    outBand = None
    out_ds = None
    print('Save to',TIFFpath,'\n')